In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import random

In [2]:
# civil_df = pd.read_csv("./data/civil_comments.csv")
civil_df_data = pd.read_csv("./data/all_data_with_identities.csv")
train_identities_df = pd.read_csv("./data/train_identities.txt", header=None)
train_identities_lists = train_identities_df.values.tolist()
test_identities_df = pd.read_csv("./data/test_identities.txt", header=None)
test_identities = test_identities_df.values.tolist()

In [3]:
train_identities = []
for ident in train_identities_lists:
    train_identities.append(ident[0])
#train_identities

In [4]:
print(sum(civil_df_data['toxicity'] < 0.5))
print(civil_df_data.shape)

397206
(448000, 59)


In [6]:
df_non_toxic = civil_df_data[civil_df_data['toxicity']<0.5]

In [7]:
df_non_toxic.shape

(397206, 59)

In [8]:
df_non_toxic['comment_text'].head()

9     Trump could accidently push a button and Canad...
10    He's considered a good candidate for a cyber-s...
11    Lela, you admit no records exist to support yo...
16         Shouldn't your handle be Republic of Uranus?
19    The guy commits espionage and the lefties at H...
Name: comment_text, dtype: object

In [9]:
len(civil_df_data)

448000

We create three lists, sentences hold the original sentence, toxicity is a value denoting how toxic the sentence was deemed, and a is the sentence with its identity token replaced with all other identity tokens. toxicity in this case will always be 0 since we are training our model using non-toxic setences.

In [10]:
sentences = []
a = []
toxicity = []

for row_index in tqdm(range(len(civil_df_data))):
    comment_text = civil_df_data.iloc[row_index]['comment_text'].split()
    if civil_df_data.iloc[row_index]['toxicity'] < 0.5 and len(comment_text) <= 10 and len(set(train_identities).intersection(comment_text)) != 0:
        identity = str(set(train_identities).intersection(comment_text).pop())
        sentences.append(civil_df_data.iloc[row_index]['comment_text'])
        toxicity.append(0)
        cur_a = []
        
        for diff_identity in train_identities:
            cur_a.append(civil_df_data.at[row_index, "comment_text"].replace(identity, diff_identity))
        a.append(cur_a)
# print(sentences)
# print(toxicity)
# print(a)


100%|██████████| 448000/448000 [02:16<00:00, 3271.44it/s]


'a' is a list of lists so this piece reformates 'a' such that it can be fed into a tensor in the right oreientation eventually. The comment_text is also appended to this df at this stage

In [11]:
civil_df_raw = pd.DataFrame(list(zip(*a)))
civil_df = civil_df_raw.T

civil_df.insert(0, column='comment_text', value=sentences)

civil_df

,comment_text,0,1,2,3,4,5,6,7,8,...,25,26,27,28,29,30,31,32,33,34
0,"No, he was accused of being a racist white man.","No, he was accused of being a racist bisexual ...","No, he was accused of being a racist transgend...","No, he was accused of being a racist trans man.","No, he was accused of being a racist queer man.","No, he was accused of being a racist lgbt man.","No, he was accused of being a racist lgbtq man.","No, he was accused of being a racist heterosex...","No, he was accused of being a racist male man.","No, he was accused of being a racist nonbinary...",...,"No, he was accused of being a racist sikh man.","No, he was accused of being a racist taoist man.","No, he was accused of being a racist older man.","No, he was accused of being a racist young man.","No, he was accused of being a racist younger man.","No, he was accused of being a racist teenage man.","No, he was accused of being a racist millenial...","No, he was accused of being a racist elderly man.","No, he was accused of being a racist blind man.","No, he was accused of being a racist deaf man."
1,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...,Only one group was carrying torches.\nNazis an...
2,Trump is a self admitted white supremacist eug...,Trump is a self admitted bisexual supremacist ...,Trump is a self admitted transgender supremaci...,Trump is a self admitted trans supremacist eug...,Trump is a self admitted queer supremacist eug...,Trump is a self admitted lgbt supremacist euge...,Trump is a self admitted lgbtq supremacist eug...,Trump is a self admitted heterosexual supremac...,Trump is a self admitted male supremacist euge...,Trump is a self admitted nonbinary supremacist...,...,Trump is a self admitted sikh supremacist euge...,Trump is a self admitted taoist supremacist eu...,Trump is a self admitted older supremacist eug...,Trump is a self admitted young supremacist eug...,Trump is a self admitted younger supremacist e...,Trump is a self admitted teenage supremacist e...,Trump is a self admitted millenial supremacist...,Trump is a self admitted elderly supremacist e...,Trump is a self admitted blind supremacist eug...,Trump is a self admitted deaf supremacist euge...
3,Your plea will fall upon deaf ears.,Your plea will fall upon bisexual ears.,Your plea will fall upon transgender ears.,Your plea will fall upon trans ears.,Your plea will fall upon queer ears.,Your plea will fall upon lgbt ears.,Your plea will fall upon lgbtq ears.,Your plea will fall upon heterosexual ears.,Your plea will fall upon male ears.,Your plea will fall upon nonbinary ears.,...,Your plea will fall upon sikh ears.,Your plea will fall upon taoist ears.,Your plea will fall upon older ears.,Your plea will fall upon young ears.,Your plea will fall upon younger ears.,Your plea will fall upon teenage ears.,Your plea will fall upon millenial ears.,Your plea will fall upon elderly ears.,Your plea will fall upon blind ears.,Your plea will fall upon deaf ears.
4,Your saying his base is a bunch of white supre...,Your saying his base is a bunch of bisexual su...,Your saying his base is a bun

In [12]:
civil_df.to_csv(Path("./data/civil_folder/civil_train_data.csv"))